In [1]:
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

In [2]:
# Beregner deltakelse ut ifra frekvensavvik med et internt effekttap på 7%
def aktivering(frekvensavvik):
    if -0.1 < frekvensavvik < 0.1:
        return frekvensavvik * 10
    
    elif frekvensavvik < -0.1:
        return -0.1 * 10 * 1.07
    
    else:
        return 0.1 * 10 * 1.07

# definere om batteriet deltar i markedet eller ikke
def deltakelse(markedspris, budpris):
    if budpris <= markedspris:
        return 1
    else:
        return 0
    
# Beregne inntekt per minutt
def inntekt_funksjon(markedspris, budpris, volum):
    deltakelse_value = deltakelse(markedspris, budpris) # [0 eller 1]
    return markedspris * deltakelse_value * volum / 60 # [EUR]/h

# Beregne kostnad per minutt
def kostnad_funksjon(volum, installert_kapasitet, aktivering, investeringskostnad, markedspris, budpris):
    kalendertap = 1/(20*365) # Beregner verditapet uten særlig aktivering med antakelse om levetid på 20år
    syklustap_value = syklustapet(volum, installert_kapasitet, aktivering) # [MWh/MWh]
    deltakelse_value = deltakelse(markedspris, budpris) # [0 eller 1]
    return (deltakelse_value * syklustap_value / 0.2 + kalendertap / (24*60)) * investeringskostnad # [EUR]


def investeringskostnad_funk(installert_kapasitet):
    return 500000 * installert_kapasitet # [EUR]

# Beregner C-rate
def C_raten(volum, installert_kapasitet, aktivering):
    return aktivering * 1.07 * volum / installert_kapasitet  # [h^-1]

def ladningen(volum, aktivering):
    spenning = 1000 # [V]
    return volum * aktivering * 1.07 / spenning # [Ah]

def syklustapet(volum, installert_kapasitet, aktivering):
    DOD_faktor = 0.0008 # [MWh/MWh]
    C_faktor = 0.3903 # [h]
    ladning_value = ladningen(volum, aktivering) # [Ah]
    C_rate_value = C_raten(volum, installert_kapasitet, aktivering) # [h^-1]
    return ladning_value * aktivering * DOD_faktor * np.exp(C_faktor * C_rate_value) # [MWh/MWh]

# Tar daglige tap og investeringskostnad som input for å beregne levetid
def levetid(pris_i_k, installert_kapasitet, kostnad, dag = True):
    if dag:
        levetid_verdi = pris_i_k * installert_kapasitet / (365 * kostnad) # [år] Overfører daglige tap til årlig tap
    else:
        levetid_verdi = pris_i_k * installert_kapasitet / kostnad # [år]
    return levetid_verdi


# Henter levetid og tar daglig inntekt som input for å beregne NNV
def NNV_investering(Inntekt, kostnad, installert_kapasitet, pris_i_k = 500000, rente = 0.05, dag = True, months = None):
    if months is None:
        levetid_verdi = levetid(pris_i_k, installert_kapasitet, kostnad, dag) # [år]

        NNV = 0 # [EUR]
        if dag:
            for i in np.arange(0, int(levetid_verdi), 1): 
                NNV += (Inntekt * 365) / ((1 + rente) ** i) # [EUR] Beregner NNV for hvert år
            # Legger til det som er igjen av inntekten i den siste måneden
            NNV += (levetid_verdi % 1) * (Inntekt * 365) / ((1 + rente) ** (levetid_verdi+1)) # [EUR]
        
        else:
            for i in np.arange(0, int(levetid_verdi), 1):
                NNV += (Inntekt) / ((1 + rente) ** i) # [EUR] Beregner NNV for hvert år
            # Legger til det som er igjen av inntekten i den siste måneden
            NNV += (levetid_verdi % 1) * (Inntekt) / ((1 + rente) ** (levetid_verdi+1)) # [EUR]

        
    else:
        Inntekt = Inntekt * (1 + (12 - months) / 12) # [EUR] Beregner hva inntekten ville vært om disse månedene var hele året
        kostnad = kostnad * (1 + (12 - months) / 12) # [EUR] Beregner hva kostnaden ville vært om disse månedene var hele året

        levetid_verdi = levetid(pris_i_k, installert_kapasitet, kostnad, dag) # [år]

        NNV = 0 # [EUR]
        for i in np.arange(0, int(levetid_verdi), 1):
            NNV += (Inntekt) / ((1 + rente) ** i) # [EUR] Beregner NNV for hvert år
        # Legger til det som er igjen av inntekten i den siste måneden
        NNV += (levetid_verdi % 1) * (Inntekt) / ((1 + rente) ** (levetid_verdi+1)) # [EUR]

    return NNV - installert_kapasitet * pris_i_k # [EUR]



def eval_optimeringsresultat_data(markedspris, frekvensavvik, budpris, volum, installert_kapasitet):
    aktivering_value = aktivering(frekvensavvik)
    investeringskostnad_value = investeringskostnad_funk(installert_kapasitet)

    inntekt_value = inntekt_funksjon(markedspris, budpris, volum)
    kostnad_value = kostnad_funksjon(volum, installert_kapasitet, aktivering_value, investeringskostnad_value, markedspris, budpris)

    return inntekt_value, kostnad_value


In [3]:
def lag_eval_pris_df(budpris_start, budpris_slutt, budpris_steg, activation_df, investeringskostnad = 500000, installert_kap = 1, diskonteringsrente = 0.05, budvolum = 2, dag = False, months = None):
    # Lag tom eval_pris-tabell på
    eval_pris = pd.DataFrame({'Budpris': np.arange(budpris_start, budpris_slutt, budpris_steg), 'NNV': 0.0})
    # Loop over alle budpriser
    for idx, budpris in enumerate(eval_pris['Budpris']):
        # Kalkuler for alle rader i activation_df
        resultater = activation_df.apply(
            lambda row: pd.Series(eval_optimeringsresultat_data(
                row['FCR-N Price EUR/MW'],
                row['Frekvensavvik'],
                budpris,
                budvolum,  # volum
                installert_kap   # installert kapasitet
            )),
            axis=1
        )

        # Pass på at eval_optimeringsresultat_data returnerer f.eks. [Inntekt, Kostnad]
        resultater.columns = ['Inntekt', 'Kostnad']

        # Summer NNV over alle rader
        NNV = NNV_investering(
            resultater['Inntekt'].sum(),
            resultater['Kostnad'].sum(),
            installert_kap,  # installert kapasitet
            investeringskostnad,  # investeringskostnad
            diskonteringsrente,  # rente
            dag,
            months
        )
        # Lagre NNV i eval_pris-tabellen
        eval_pris.at[idx, 'NNV'] = NNV
    return eval_pris

def lag_eval_volum_df(budvolum_start, budvolum_slutt, budvolum_steg, activation_df, investeringskostnad = 500000, installert_kap = 1, diskonteringsrente = 0.05, budpris = 49.24, dag = False, months = None):
    # Lag tom eval_pris-tabell på
    eval_volum = pd.DataFrame({'Innbydd kapasitet': np.arange(budvolum_start, budvolum_slutt, budvolum_steg), 'NNV': 0.0})

    # Loop over alle budpriser
    for idx, budvolum in enumerate(eval_volum['Innbydd kapasitet']):
        # Kalkuler for alle rader i activation_df
        resultater = activation_df.apply(
            lambda row: pd.Series(eval_optimeringsresultat_data(
                row['FCR-N Price EUR/MW'],
                row['Frekvensavvik'],
                budpris,
                budvolum,  # volum
                installert_kap   # installert kapasitet
            )),
            axis=1
        )

        # Pass på at eval_optimeringsresultat_data returnerer f.eks. [Inntekt, Kostnad]
        resultater.columns = ['Inntekt', 'Kostnad']

        # Summer NNV over alle rader
        NNV = NNV_investering(
            resultater['Inntekt'].sum(),
            resultater['Kostnad'].sum(),
            installert_kap,  # installert kapasitet
            investeringskostnad,  # investeringskostnad
            diskonteringsrente,  # rente
            dag,
            months
        )
        # Lagre NNV i eval_pris-tabellen
        eval_volum.at[idx, 'NNV'] = NNV
    return eval_volum

def lag_eval_kapasitet_df(installert_kap_start, installert_kap_slutt, installert_kap_steg, activation_df, investeringskostnad = 500000, budvolum = 2, diskonteringsrente = 0.05, budpris = 49.24, dag = False, months = None):
    # Lag tom eval_pris-tabell på
    eval_kapasitet = pd.DataFrame({'Installert kapasitet': np.arange(installert_kap_start, installert_kap_slutt, installert_kap_steg), 'NNV': 0.0})

    # Loop over alle budpriser
    for idx, installert_kap in enumerate(eval_kapasitet['Installert kapasitet']):
        # Kalkuler for alle rader i activation_df
        resultater = activation_df.apply(
            lambda row: pd.Series(eval_optimeringsresultat_data(
                row['FCR-N Price EUR/MW'],
                row['Frekvensavvik'],
                budpris,
                budvolum,  # volum
                installert_kap   # installert kapasitet
            )),
            axis=1
        )
        # Pass på at eval_optimeringsresultat_data returnerer f.eks. [Inntekt, Kostnad]
        resultater.columns = ['Inntekt', 'Kostnad']

                # Summer NNV over alle rader
        NNV = NNV_investering(
            resultater['Inntekt'].sum(),
            resultater['Kostnad'].sum(),
            installert_kap,  # installert kapasitet
            investeringskostnad,  # investeringskostnad
            diskonteringsrente,  # rente
            dag,
            months
        )
        # Lagre NNV i eval_kap
        eval_kapasitet.at[idx, 'NNV'] = NNV
    return eval_kapasitet

def lag_levetid_volum_df(budvolum_start, budvolum_slutt, budvolum_steg, activation_df, investeringskostnad = 500000, installert_kap = 1, diskonteringsrente = 0.05, budpris = 49.24, dag = False, months = None):
    # Lag tom eval_pris-tabell på
    levetid_volum = pd.DataFrame({'Innbydd kapasitet': np.arange(budvolum_start, budvolum_slutt, budvolum_steg), 'Levetid': 0.0})

    # Loop over alle budvolumer
    for idx, budvolum in enumerate(levetid_volum['Innbydd kapasitet']):
        # Kalkuler for alle rader i activation_df
        resultater = activation_df.apply(
            lambda row: pd.Series(eval_optimeringsresultat_data(
                row['FCR-N Price EUR/MW'],
                row['Frekvensavvik'],
                budpris,
                budvolum,  # volum
                installert_kap   # installert kapasitet
            )),
            axis=1
        )

        # Pass på at eval_optimeringsresultat_data returnerer f.eks. [Inntekt, Kostnad]
        resultater.columns = ['Inntekt', 'Kostnad']

        levetiden = levetid(500000, 1, resultater['Kostnad'].sum(), dag= False)

        # Lagre levetid i levetid_volum-tabellen
        levetid_volum.at[idx, 'Levetid'] = levetiden
    return levetid_volum

def lag_inntekt_volum_df(budvolum_start, budvolum_slutt, budvolum_steg, activation_df, investeringskostnad = 500000, installert_kap = 1, diskonteringsrente = 0.05, budpris = 49.24, dag = False, months = None):
    # Lag tom eval_pris-tabell på
    levetid_volum = pd.DataFrame({'Innbydd kapasitet': np.arange(budvolum_start, budvolum_slutt, budvolum_steg), 'Inntekt': 0.0})

    # Loop over alle budvolumer
    for idx, budvolum in enumerate(levetid_volum['Innbydd kapasitet']):
        # Kalkuler for alle rader i activation_df
        resultater = activation_df.apply(
            lambda row: pd.Series(eval_optimeringsresultat_data(
                row['FCR-N Price EUR/MW'],
                row['Frekvensavvik'],
                budpris,
                budvolum,  # volum
                installert_kap   # installert kapasitet
            )),
            axis=1
        )

        # Pass på at eval_optimeringsresultat_data returnerer f.eks. [Inntekt, Kostnad]
        resultater.columns = ['Inntekt', 'Kostnad']

        # Lagre levetid i levetid_volum-tabellen
        levetid_volum.at[idx, 'Inntekt'] = resultater['Inntekt'].sum()
    return levetid_volum

In [4]:
# read the csv file into a pandas dataframe split on ; and skip bad lines
PrimRes = pd.read_csv('/Users/bendikostvik/Library/CloudStorage/OneDrive-NorwegianUniversityofLifeSciences/Documents/Skole/Master/data/PrimaryReservesD-1-2024_test.csv', sep=';', on_bad_lines='skip')

# filter on NO3 in Area
NO3 = PrimRes[PrimRes['Area'] == 'NO3']

NO3.drop(columns=['FCR-N Price NOK/MW', 'Area', 'Hournumber', 'FCR-D Price EUR/MW', 'FCR-D Price NOK/MW', 'FCR-D Volume MW'], inplace=True)
# convert the Time(Local) column to a datetime object
NO3['Time(Local)'] = pd.to_datetime(NO3['Time(Local)'], dayfirst=True)

# convert the FCR-N Price EUR/MW column to a float
NO3['FCR-N Price EUR/MW'] = NO3['FCR-N Price EUR/MW'].str.replace(',', '.').astype('float32')

# set the 'Time(Local)' column as the index
NO3.set_index('Time(Local)', inplace=True)
NO3.index = pd.to_datetime(NO3.index, utc=True)  # Ensure the index is datetime

NO3_min = NO3.resample('1T').ffill()
NO3_min.index = NO3_min.index.tz_convert('UTC')

# read '/Users/bendikostvik/Desktop/frekvensdata/frekvensdata2024_min.csv'
frekvensdata = pd.read_csv('/Users/bendikostvik/Desktop/frekvensdata/frekvensdata2024_min.csv')
# make the frekvensdata a pandas df
frekvensdata = pd.DataFrame(frekvensdata)
frekvensdata['Time'] = pd.to_datetime(frekvensdata['Time'])
frekvensdata['Value'] = frekvensdata['Value'].apply(lambda x: 50 if x < 10 else x)
frekvensdata['Frekvensavvik'] = frekvensdata['Value'] - 50

# set the 'Time' column as the index
frekvensdata.set_index('Time', inplace=True)
frekvensdata.index = pd.to_datetime(frekvensdata.index, utc=True) 

# Merge the two dataframes on the index
activation_df = pd.merge(frekvensdata, NO3_min, left_index=True, right_index=True)
# give the index name: Time
activation_df.index.name = 'Time'

/var/folders/ky/j9w4hxz113l4cjnkxpsycpf40000gq/T/ipykernel_62535/3243529457.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NO3.drop(columns=['FCR-N Price NOK/MW', 'Area', 'Hournumber', 'FCR-D Price EUR/MW', 'FCR-D Price NOK/MW', 'FCR-D Volume MW'], inplace=True)
/var/folders/ky/j9w4hxz113l4cjnkxpsycpf40000gq/T/ipykernel_62535/3243529457.py:9: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  NO3['Time(Local)'] = pd.to_datetime(NO3['Time(Local)'], dayfirst=True)
/var/folders/ky/j9w4hxz113l4cjnkxpsycpf40000gq/T/ipykernel_62535/32435

## Årstider og lønnsomhet

In [5]:
activation_df[['Inntekt', 'Kostnad']] = activation_df.apply(
    lambda row: pd.Series(eval_optimeringsresultat_data(
        row['FCR-N Price EUR/MW'],  # markedspris
        row['Frekvensavvik'],       # frekvensavvik
        15,                      # budpris
        2.5,                          # volum
        2                           # installert_kapasitet
    )),
    axis=1
)


In [ ]:
months = activation_df.resample('M').sum()
# rename the months to Januar, Februar, etc.
months.index = months.index.strftime('%B')

In [6]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=months.index, y=months['Inntekt'], mode='lines', name='Inntekt'))
fig.add_trace(go.Scatter(x=months.index, y=months['Kostnad'], mode='lines', name='Kostnad'))

fig.update_layout(#title='Lønnsomhet for hver måned',
                  xaxis_title='Måned',
                  yaxis_title='EUR')
fig.show()

NameError: name 'months' is not defined

### Sommer

In [ ]:
df_sommer = activation_df[activation_df.index.month.isin([4, 5, 6, 7, 8])]
eval_pris_sommer = lag_eval_pris_df( 30, 70, 2, df_sommer, months = 5, budvolum = 2)
eval_volum_sommer = lag_eval_volum_df(1, 7, 0.5, df_sommer, months = 5, budpris = 36)

In [ ]:
# plot eval_pris og eval_volum
fig = make_subplots(rows=1, cols=2, subplot_titles=("Budpris", "Innbydd effekt"), shared_yaxes=True, horizontal_spacing=0.03)
# add a black dash line at y = 0
fig.add_shape(type="line",
              x0=-2, x1=max(eval_pris_sommer['Budpris']),
              y0=0, y1=0,
              line=dict(color="grey", width=2), row=1, col=1)
fig.add_shape(type="line",
              x0=min(eval_volum_sommer['Innbydd kapasitet']), x1=max(eval_volum_sommer['Innbydd kapasitet']),
              y0=0, y1=0,
              line=dict(color="grey", width=2), row=1, col=2)
# plot eval_pris
fig.add_trace(go.Scatter(x=eval_pris_sommer['Budpris'], y=eval_pris_sommer['NNV'], mode='lines', name='NNV', line=dict(color='blue')), row=1, col=1)
fig.update_layout(showlegend=False)
# plot eval_volum
fig.add_trace(go.Scatter(x=eval_volum_sommer['Innbydd kapasitet'], y=eval_volum_sommer['NNV'], mode='lines', name='NNV', line=dict(color='blue')), row=1, col=2)
# Add a vertical line and shade the area out of range for the second column
fig.update_layout(#title='Lønnsomhet mot Budpris, Installert kapasitet and Volum',
                  yaxis_title='NNV [EUR]',
                  xaxis_title='Budpris [EUR]',
                  xaxis2_title='Innbydd kapasitet [MW]',
                  showlegend=False)  # Hide legends
fig.show()

### Vinter

In [ ]:
df_vinter = activation_df[activation_df.index.month.isin([1, 2, 3, 9, 10, 11, 12])]
eval_pris_vinter = lag_eval_pris_df( 15, 70, 5, df_vinter, months = 7, budvolum = 2)
eval_volum_vinter = lag_eval_volum_df(1, 7, 0.5, df_vinter, months = 7, budpris=20)

In [ ]:



fig = make_subplots(rows=1, cols=2, subplot_titles=("Budpris", "Budvolum"), shared_yaxes=True, horizontal_spacing=0.03)

# add a black dash line at y = 0
fig.add_shape(type="line",
              x0=min(eval_pris_vinter['Budpris']), x1=max(eval_pris_vinter['Budpris']),
              y0=-2, y1=0,
              line=dict(color="grey", width=2), row=1, col=1)
fig.add_shape(type="line",
              x0=-2, x1=max(eval_volum_vinter['Innbydd kapasitet']),
              y0=-2, y1=0,
              line=dict(color="grey", width=2), row=1, col=2)


# plot eval_pris
fig.add_trace(go.Scatter(x=eval_pris_vinter['Budpris'], y=eval_pris_vinter['NNV'], mode='lines', name='NNV', line=dict(color='blue')), row=1, col=1)

# plot eval_volum
fig.add_trace(go.Scatter(x=eval_volum_vinter['Innbydd kapasitet'], y=eval_volum_vinter['NNV'], mode='lines', name='NNV', line=dict(color='blue')), row=1, col=2)

fig.update_layout(#title='Lønnsomhet mot Budpris, Installert kapasitet and Volum',
                  yaxis_title='NNV [EUR]',
                  xaxis_title='Budpris [EUR]',
                  xaxis2_title='Innbydd kapasitet [MW]',
                  showlegend=False)  # Hide legends
fig.show()

### NNV mot effekt

In [ ]:
nnv_plot_df = lag_eval_volum_df(0,8,1, 
                                activation_df, 
                                budpris = 30, 
                                installert_kap = 1, 
                                investeringskostnad = 500000, 
                                diskonteringsrente = 0.05, 
                                dag = False)

In [ ]:
# plot nnv mot volum
fig = go.Figure()
fig.add_trace(go.Scatter(x=nnv_plot_df['Innbydd kapasitet'], y=nnv_plot_df['NNV'], mode='lines', name='NNV'))
fig.update_layout(
                  xaxis_title='Innbydd kapasitet [MW]',
                  yaxis_title='NNV [EUR]')
fig.show()

### Levetid og Inntekt mot effekt

In [ ]:
levetid_volum = lag_levetid_volum_df(0,8,1, activation_df, budpris = 30)
Inntekten_volum = lag_inntekt_volum_df(0,8,1, activation_df, budpris = 30)

In [ ]:
go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x = levetid_volum['Innbydd kapasitet'], y = levetid_volum['Levetid'], mode='lines', name='Levetid'), secondary_y=False)
fig.add_trace(go.Scatter(x = Inntekten_volum['Innbydd kapasitet'], y = Inntekten_volum['Inntekt'], mode='lines', name='Inntekt'), secondary_y=True)

# update yaxis properties
fig.update_yaxes(title_text="Levetid [År]", secondary_y=False)
fig.update_yaxes(title_text="Inntekt [EUR/År]", secondary_y=True)
fig.update_xaxes(title_text="Innbydd kapasitet [MW]")
fig.show()